In [1]:
import numpy as np
from qiskit import *
from math import pi, log10
import matplotlib.pyplot as plt
%matplotlib inline
from qiskit.circuit import *
from qiskit import IBMQ
from qiskit.tools.monitor import job_monitor
from scipy.optimize import *
from copy import deepcopy
import time

In [2]:
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [3]:
n=3
k=1

In [4]:
circuit=QuantumCircuit(n+k+1)

In [5]:
circuit.h(k+1)
for i in range(k+2,n+k+1):
    circuit.cx(k+1,i)
circuit.h(1)
for i in range(2,k+1):
    circuit.cx(1,i)
circuit.barrier(range(k+1,n+k+1))

In [6]:
circuit.draw()

q_0: ──────────────────
     ┌───┐             
q_1: ┤ H ├─────────────
     ├───┤           ░ 
q_2: ┤ H ├──■────■───░─
     └───┘┌─┴─┐  │   ░ 
q_3: ─────┤ X ├──┼───░─
          └───┘┌─┴─┐ ░ 
q_4: ──────────┤ X ├─░─
               └───┘ ░

In [7]:
#params=[Parameter('theta'),Parameter('B'),Parameter('C')]
string="["
ng=0
for i in range(n*(n+1)-1):
    string=string+"[Parameter(\'theta_"+str(ng)+"\'),Parameter(\'phi_"+str(ng)+"\'),Parameter(\'lamda_"+str(ng)+"\')],"
    ng+=1
string=string+"[Parameter(\'theta_"+str(ng)+"\'),Parameter(\'phi_"+str(ng)+"\'),Parameter(\'lamda_"+str(ng)+"\')]]"

params=eval(string)

#print(params)

U_AB=QuantumCircuit(n,name='U_AB')
nu=0
for i in range(n):
    U_AB.u3(params[nu][0],params[nu][1],params[nu][2],i)
    nu+=1

U_AB.barrier(range(n))
    
for i in range(n):
    for j in range(n):
        if j!=i:
            U_AB.cu3(params[nu][0],params[nu][1],params[nu][2],i,j)
            nu+=1
        
U_AB.barrier(range(n))
        
for i in range(n):
    U_AB.u3(params[nu][0],params[nu][1],params[nu][2],i)
    nu+=1

U_inst=U_AB.to_instruction()
U_AB.draw()

┌───────────────────────────┐ ░                              »
q_0: ┤ U3(theta_0,phi_0,lamda_0) ├─░───────────────■──────────────»
     ├───────────────────────────┤ ░ ┌─────────────┴─────────────┐»
q_1: ┤ U3(theta_1,phi_1,lamda_1) ├─░─┤ U3(theta_3,phi_3,lamda_3) ├»
     ├───────────────────────────┤ ░ └───────────────────────────┘»
q_2: ┤ U3(theta_2,phi_2,lamda_2) ├─░──────────────────────────────»
     └───────────────────────────┘ ░                              »
«                                  ┌───────────────────────────┐»
«q_0: ──────────────■──────────────┤ U3(theta_5,phi_5,lamda_5) ├»
«                   │              └─────────────┬─────────────┘»
«q_1: ──────────────┼────────────────────────────■──────────────»
«     ┌─────────────┴─────────────┐                             »
«q_2: ┤ U3(theta_4,phi_4,lamda_4) ├─────────────────────────────»
«     └───────────────────────────┘                             »
«                                  ┌───────────────────────────┐»
«q_0: ─────────────────────────────┤ U3(theta_7,phi_7,lamda_7) ├»
«                                  └─────────────┬─────────────┘»
«q_1: ──────────────■────────────────────────────┼──────────────»
«     ┌─────────────┴─────────────┐              │              »
«q_2: ┤ U3(theta_6,phi_6,lamda_6) ├──────────────■──────────────»
«     └───────────────────────────┘                             »
«                                   ░  ┌───────────────────────────┐  
«q_0: ──────────────────────────────░──┤ U3(theta_9,phi_9,lamda_9) ├──
«     ┌───────────────────────────┐ ░ ┌┴───────────────────────────┴─┐
«q_1: ┤ U3(theta_8,phi_8,lamda_8) ├─░─┤ U3(theta_10,phi_10,lamda_10) ├
«     └─────────────┬─────────────┘ ░ ├──────────────────────────────┤
«q_2: ──────────────■───────────────░─┤ U3(theta_11,phi_11,lamda_11) ├
«                                   ░ └──────────────────────────────┘

In [8]:
circuit.append(U_inst,range(k+1,n+k+1))
#circuit.draw()
circuit.barrier(range(2*k+1,n+k+1))
circuit.h(0)
for i in range(k):
    circuit.cswap(0,i+1,i+k+1)
circuit.h(0)
meas=QuantumCircuit(n+k+1,1)
# meas.measure(range(5,8),range(3))
meas.measure(0,0)
qc=circuit+meas
qc.draw()

┌───┐             »
q_0: ┤ H ├─────────────»
     ├───┤             »
q_1: ┤ H ├─────────────»
     ├───┤           ░ »
q_2: ┤ H ├──■────■───░─»
     └───┘┌─┴─┐  │   ░ »
q_3: ─────┤ X ├──┼───░─»
          └───┘┌─┴─┐ ░ »
q_4: ──────────┤ X ├─░─»
               └───┘ ░ »
c_0: ══════════════════»
                       »
«                                                                                                                                                                                                                                                                                             »
«q_0: ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
«                                                                                                                                                                                                                                                                                             »
«q_1: ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
«     ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐»
«q_2: ┤0                                                                                                                                                                                                                                                                                     ├»
«     │                                                                                                                                                                                                                                                                                      │»
«q_3: ┤1 U_ab(lamda_0,lamda_1,lamda_10,lamda_11,lamda_2,lamda_3,lamda_4,lamda_5,lamda_6,lamda_7,lamda_8,lamda_9,phi_0,phi_1,phi_10,phi_11,phi_2,phi_3,phi_4,phi_5,phi_6,phi_7,phi_8,phi_9,theta_0,theta_1,theta_10,theta_11,theta_2,theta_3,theta_4,theta_5,theta_6,theta_7,theta_8,theta_9) ├»
«     │                                                                                                                                                                                                                                                                                      │»
«q_4: ┤2                                                                                                                                                                                                                                                                                     ├»
«     └──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘»
«c_0: ════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════»
«                                                                                                                                                                                                                               

In [9]:
name2array={}
name2string={}
array2string={}

def register_string(name):
    string2="{"
    ii=0
    for i in range(n*(n+1)-1):
        for j in range(3):
            string2=string2+"params["+str(i)+"]["+str(j)+"]:name2array[\""+name+"\"]["+str(ii)+"],"
            ii+=1
    string2=string2+"params["+str(n*(n+1)-1)+"][0]:name2array[\""+name+"\"]["+str(ii)+"],"
    ii+=1
    string2=string2+"params["+str(n*(n+1)-1)+"][1]:name2array[\""+name+"\"]["+str(ii)+"],"
    ii+=1
    string2=string2+"params["+str(n*(n+1)-1)+"][2]:name2array[\""+name+"\"]["+str(ii)+"]}"
    name2string[name]=string2
    
def array_to_string(array_name):
    string3="{"
    ii=0
    for i in range(n*(n+1)-1):
        for j in range(3):
            string3=string3+"params["+str(i)+"]["+str(j)+"]:"+array_name+"["+str(ii)+"],"
            ii+=1
    string3=string3+"params["+str(n*(n+1)-1)+"][0]:"+array_name+"["+str(ii)+"],"
    ii+=1
    string3=string3+"params["+str(n*(n+1)-1)+"][1]:"+array_name+"["+str(ii)+"],"
    ii+=1
    string3=string3+"params["+str(n*(n+1)-1)+"][2]:"+array_name+"["+str(ii)+"]}"
    return string3
    
global_string=array_to_string("x_in_function")    
    
register_string("x")
register_string("xa")
register_string("xb")

In [10]:
def assign_params(array_name):

    dictionary=eval(name2string[array_name])

    bound_circuit=qc.bind_parameters(dictionary)

    backend_sim=Aer.get_backend('qasm_simulator')

    nshots=1024
    job_sim=execute(bound_circuit,backend_sim,shots=nshots)
    
    result_sim=job_sim.result()

    counts=result_sim.get_counts(bound_circuit)

    c2_func=2*counts['0']/nshots-1
    func=log10(1-c2_func)
    
    return func

In [12]:
def func(x):
    name2array["x"] = x
    f=assign_params("x")
    del name2array["x"]
    return f

def funcd(x):
    name2array["x"] = x
    f=assign_params("x")
    
    h=1e-8
    df=np.zeros(n*(n+1)*3)
    name2array["xa"]=deepcopy(x)
    name2array["xb"]=deepcopy(x)
    jj=0
    for i in range(n*(n+1)):
        for j in range(3):
            name2array["xa"][jj]+=h/2
            name2array["xb"][jj]-=h/2
            df[jj]=(assign_params("xa")-assign_params("xb"))/h
            name2array["xa"][jj]-=h/2
            name2array["xb"][jj]+=h/2
            jj+=1
    del name2array["xa"]
    del name2array["xb"]
    del name2array["x"]
    return f,df

def faster_funcd(x):
    x_in_function=deepcopy(x)
    dictionary=eval(global_string)
    binds=[]
    binds.append(dictionary)
    h=1e-8
    for i in range(n*(n+1)):
        for j in range(3):
            dictionary[params[i][j]]+=h/2
            binds.append(dictionary)
            dictionary[params[i][j]]-=h
            binds.append(dictionary)
            dictionary[params[i][j]]+=h/2
    nshots=1024
    job = execute(qc,backend=Aer.get_backend('qasm_simulator'),parameter_binds=binds,shots=nshots)
    counts = [job.result().get_counts(i) for i in range(len(job.result().results))]

    f=log10(1-(2*counts[0]['0']/nshots-1))
    
    df=np.zeros(n*(n+1)*3)
    for i in range(n*(n+1)*3):
        df[i]=(log10(1-(2*counts[2*i+1]['0']/nshots-1))-log10(1-(2*counts[2*i+2]['0']/nshots-1)))/h
    
    return f,df
    
def funcd_quantum(x):
    x_in_function=deepcopy(x)
    dictionary=eval(global_string)
    binds=[]
    binds.append(dictionary)
    h=1e-8
    for i in range(n*(n+1)):
        for j in range(3):
            dictionary[params[i][j]]+=h/2
            binds.append(dictionary)
            dictionary[params[i][j]]-=h
            binds.append(dictionary)
            dictionary[params[i][j]]+=h/2
    
    provider = IBMQ.get_provider(group='open')
    backend=provider.get_backend('ibmq_essex')
    nshots=1024
    job = execute(qc,backend=backend,parameter_binds=binds,shots=nshots)
    job_monitor(job)
    counts = [job.result().get_counts(i) for i in range(len(job.result().results))]

    f=log10(1-(2*counts[0]['0']/nshots-1))
    
    df=np.zeros(n*(n+1)*3)
    for i in range(n*(n+1)*3):
        df[i]=(log10(1-(2*counts[2*i+1]['0']/nshots-1))-log10(1-(2*counts[2*i+2]['0']/nshots-1)))/h
    
    return f,df
    

xm=np.ones(n*(n+1)*3)*2*pi
for i in range(n*(n+1)):
    xm[3*i]=pi

class MyBounds(object):
    def __init__(self, xmax=xm, xmin=np.zeros(n*(n+1)*3) ):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x < self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin

In [14]:
x_test=np.ones(n*(n+1)*3)*pi/2
register_string("x_test")
time_start1=time.time()
f1=funcd(x_test)
time_end1=time.time()
print('time_funcd cost: ','%.3f' %(time_end1-time_start1),' s')

time_start2=time.time()
f2=faster_funcd(x_test)
time_end2=time.time()
print('time_faster cost: ','%.3f' %(time_end2-time_start2),' s')

time_start3=time.time()
f3=funcd_quantum(x_test)
time_end3=time.time()
print('time_quantum cost: ','%.3f' %(time_end3-time_start3),' s')
print("funcd = ",'%.6f' %(f1[0]))
print("faster_funcd = ",'%.6f' %(f2[0]))
print("funcd_quantum = ",'%.6f' %(f3[0]))

time_funcd cost:  2.009  s
time_faster cost:  1.342  s
Job Status: job has successfully run
time_quantum cost:  180.807  s
funcd =  -0.338202
faster_funcd =  -0.340054
funcd_quantum =  -0.233599


In [15]:
minimizer_kwargs = {"method":"L-BFGS-B", "jac":True}
mybounds = MyBounds()
x0=np.ones(n*(n+1)*3)*pi/2
register_string("x0")

#ret = basinhopping(funcd, x0, minimizer_kwargs=minimizer_kwargs,niter=3000,accept_test=mybounds,disp=True)
#ret = basinhopping(func, x0,niter=3000,accept_test=mybounds,disp=True)
ret = basinhopping(faster_funcd, x0, minimizer_kwargs=minimizer_kwargs,niter=3000,accept_test=mybounds,disp=True)

print(ret.x)
print(ret.fun)

basinhopping step 0: f -0.290969
basinhopping step 1: f -0.32188 trial_f -0.32188 accepted 1  lowest_f -0.32188
found new global minimum on step 1 with function value -0.32188
basinhopping step 2: f -0.249877 trial_f -0.249877 accepted 1  lowest_f -0.32188
basinhopping step 3: f -0.387051 trial_f -0.387051 accepted 1  lowest_f -0.387051
found new global minimum on step 3 with function value -0.387051
basinhopping step 4: f -0.193396 trial_f -0.193396 accepted 1  lowest_f -0.387051
basinhopping step 5: f -0.111575 trial_f -0.111575 accepted 1  lowest_f -0.387051
basinhopping step 6: f -0.111575 trial_f -0.432808 accepted 0  lowest_f -0.387051
basinhopping step 7: f -0.111575 trial_f -0.248372 accepted 0  lowest_f -0.387051
basinhopping step 8: f -0.340054 trial_f -0.340054 accepted 1  lowest_f -0.387051
basinhopping step 9: f -0.430516 trial_f -0.430516 accepted 1  lowest_f -0.430516
found new global minimum on step 9 with function value -0.430516
basinhopping step 10: f -0.374816 trial

basinhopping step 87: f -0.242402 trial_f -0.162727 accepted 0  lowest_f -0.61236
basinhopping step 88: f -0.276301 trial_f -0.276301 accepted 1  lowest_f -0.61236
basinhopping step 89: f -0.276301 trial_f -0.294297 accepted 0  lowest_f -0.61236
basinhopping step 90: f -0.276301 trial_f -0.351335 accepted 0  lowest_f -0.61236
basinhopping step 91: f -0.276301 trial_f -0.374816 accepted 0  lowest_f -0.61236
basinhopping step 92: f -0.276301 trial_f -0.150561 accepted 0  lowest_f -0.61236
basinhopping step 93: f -0.276301 trial_f -0.108297 accepted 0  lowest_f -0.61236
basinhopping step 94: f -0.276301 trial_f -0.0943727 accepted 0  lowest_f -0.61236
basinhopping step 95: f -0.276301 trial_f -0.21234 accepted 0  lowest_f -0.61236
basinhopping step 96: f -0.226396 trial_f -0.226396 accepted 1  lowest_f -0.61236
basinhopping step 97: f -0.226396 trial_f -0.70927 accepted 0  lowest_f -0.61236
basinhopping step 98: f -0.226396 trial_f -0.226396 accepted 1  lowest_f -0.61236
basinhopping step

basinhopping step 176: f -0.202765 trial_f -0.16894 accepted 0  lowest_f -0.61236
basinhopping step 177: f -0.202765 trial_f -0.0901766 accepted 0  lowest_f -0.61236
basinhopping step 178: f -0.202765 trial_f -0.271519 accepted 0  lowest_f -0.61236
basinhopping step 179: f -0.202765 trial_f -0.144604 accepted 0  lowest_f -0.61236
basinhopping step 180: f -0.202765 trial_f -0.307869 accepted 0  lowest_f -0.61236
basinhopping step 181: f -0.202765 trial_f -0.17907 accepted 0  lowest_f -0.61236
basinhopping step 182: f -0.202765 trial_f -0.194722 accepted 0  lowest_f -0.61236
basinhopping step 183: f -0.202765 trial_f -0.287666 accepted 0  lowest_f -0.61236
basinhopping step 184: f -0.209583 trial_f -0.209583 accepted 1  lowest_f -0.61236
basinhopping step 185: f -0.209583 trial_f -0.351335 accepted 0  lowest_f -0.61236
basinhopping step 186: f -0.209583 trial_f -0.136398 accepted 0  lowest_f -0.61236
basinhopping step 187: f -0.239448 trial_f -0.239448 accepted 1  lowest_f -0.61236
basin

basinhopping step 262: f -0.355162 trial_f -0.382934 accepted 0  lowest_f -0.61236
basinhopping step 263: f -0.355162 trial_f -0.340054 accepted 0  lowest_f -0.61236
basinhopping step 264: f -0.338202 trial_f -0.338202 accepted 1  lowest_f -0.61236
basinhopping step 265: f -0.338202 trial_f -0.294297 accepted 0  lowest_f -0.61236
basinhopping step 266: f -0.338202 trial_f -0.338202 accepted 0  lowest_f -0.61236
basinhopping step 267: f -0.338202 trial_f -0.338202 accepted 0  lowest_f -0.61236
basinhopping step 268: f -0.338202 trial_f -0.374816 accepted 0  lowest_f -0.61236
basinhopping step 269: f -0.338202 trial_f -0.287666 accepted 0  lowest_f -0.61236
basinhopping step 270: f -0.338202 trial_f -0.38089 accepted 0  lowest_f -0.61236
basinhopping step 271: f -0.368826 trial_f -0.368826 accepted 1  lowest_f -0.61236
basinhopping step 272: f -0.368826 trial_f -0.336358 accepted 0  lowest_f -0.61236
basinhopping step 273: f -0.368826 trial_f -0.347542 accepted 0  lowest_f -0.61236
basin

basinhopping step 350: f -0.637388 trial_f -0.539008 accepted 0  lowest_f -0.70927
basinhopping step 351: f -0.637388 trial_f -0.497082 accepted 0  lowest_f -0.70927
basinhopping step 352: f -0.637388 trial_f -0.553934 accepted 0  lowest_f -0.70927
basinhopping step 353: f -0.45885 trial_f -0.45885 accepted 1  lowest_f -0.70927
basinhopping step 354: f -0.3933 trial_f -0.3933 accepted 1  lowest_f -0.70927
basinhopping step 355: f -0.3933 trial_f -0.421468 accepted 0  lowest_f -0.70927
basinhopping step 356: f -0.437428 trial_f -0.437428 accepted 1  lowest_f -0.70927
basinhopping step 357: f -0.437428 trial_f -0.50515 accepted 0  lowest_f -0.70927
basinhopping step 358: f -0.437428 trial_f -0.412605 accepted 0  lowest_f -0.70927
basinhopping step 359: f -0.355162 trial_f -0.355162 accepted 1  lowest_f -0.70927
basinhopping step 360: f -0.345658 trial_f -0.345658 accepted 1  lowest_f -0.70927
basinhopping step 361: f -0.345658 trial_f -0.277906 accepted 0  lowest_f -0.70927
basinhopping 

...

basinhopping step 2622: f -0.121559 trial_f -0.121559 accepted 1  lowest_f -1.17779
basinhopping step 2623: f -0.121559 trial_f -0.0986098 accepted 0  lowest_f -1.17779
basinhopping step 2624: f -0.190756 trial_f -0.190756 accepted 1  lowest_f -1.17779
basinhopping step 2625: f -0.190756 trial_f -0.0933199 accepted 0  lowest_f -1.17779
basinhopping step 2626: f -0.143422 trial_f -0.143422 accepted 1  lowest_f -1.17779
basinhopping step 2627: f -0.130631 trial_f -0.130631 accepted 1  lowest_f -1.17779
basinhopping step 2628: f -0.130631 trial_f -0.138727 accepted 0  lowest_f -1.17779
basinhopping step 2629: f -0.155387 trial_f -0.155387 accepted 1  lowest_f -1.17779
basinhopping step 2630: f -0.142244 trial_f -0.142244 accepted 1  lowest_f -1.17779
basinhopping step 2631: f -0.166445 trial_f -0.166445 accepted 1  lowest_f -1.17779
basinhopping step 2632: f -0.162727 trial_f -0.162727 accepted 1  lowest_f -1.17779
basinhopping step 2633: f -0.20412 trial_f -0.20412 accepted 1  lowest_f -

basinhopping step 2707: f -0.196052 trial_f -0.233599 accepted 0  lowest_f -1.17779
basinhopping step 2708: f -0.196052 trial_f -0.186826 accepted 0  lowest_f -1.17779
basinhopping step 2709: f -0.196052 trial_f -0.197387 accepted 0  lowest_f -1.17779
basinhopping step 2710: f -0.196052 trial_f -0.237978 accepted 0  lowest_f -1.17779
basinhopping step 2711: f -0.196052 trial_f -0.243887 accepted 0  lowest_f -1.17779
basinhopping step 2712: f -0.196052 trial_f -0.26679 accepted 0  lowest_f -1.17779
basinhopping step 2713: f -0.196052 trial_f -0.17907 accepted 0  lowest_f -1.17779
basinhopping step 2714: f -0.196052 trial_f -0.229263 accepted 0  lowest_f -1.17779
basinhopping step 2715: f -0.196052 trial_f -0.428237 accepted 0  lowest_f -1.17779
basinhopping step 2716: f -0.196052 trial_f -0.170194 accepted 0  lowest_f -1.17779
basinhopping step 2717: f -0.196052 trial_f -0.185523 accepted 0  lowest_f -1.17779
basinhopping step 2718: f -0.196052 trial_f -0.20412 accepted 0  lowest_f -1.1

basinhopping step 2796: f -0.533179 trial_f -0.421468 accepted 0  lowest_f -1.17779
basinhopping step 2797: f -0.533179 trial_f -0.524579 accepted 0  lowest_f -1.17779
basinhopping step 2798: f -0.533179 trial_f -0.675846 accepted 0  lowest_f -1.17779
basinhopping step 2799: f -0.533179 trial_f -0.499755 accepted 0  lowest_f -1.17779
adaptive stepsize: acceptance rate 0.492143 target 0.500000 new stepsize 0.265721 old stepsize 0.295245
basinhopping step 2800: f -0.533179 trial_f -0.343782 accepted 0  lowest_f -1.17779
basinhopping step 2801: f -0.533179 trial_f -0.432808 accepted 0  lowest_f -1.17779
basinhopping step 2802: f -0.533179 trial_f -0.37281 accepted 0  lowest_f -1.17779
basinhopping step 2803: f -0.533179 trial_f -0.476274 accepted 0  lowest_f -1.17779
basinhopping step 2804: f -0.533179 trial_f -0.524579 accepted 0  lowest_f -1.17779
basinhopping step 2805: f -0.533179 trial_f -0.456417 accepted 0  lowest_f -1.17779
basinhopping step 2806: f -0.533179 trial_f -0.556982 acc

basinhopping step 2880: f -0.141068 trial_f -0.0737862 accepted 0  lowest_f -1.17779
basinhopping step 2881: f -0.141068 trial_f -0.150561 accepted 0  lowest_f -1.17779
basinhopping step 2882: f -0.141068 trial_f -0.0839575 accepted 0  lowest_f -1.17779
basinhopping step 2883: f -0.115984 trial_f -0.115984 accepted 1  lowest_f -1.17779
basinhopping step 2884: f -0.106126 trial_f -0.106126 accepted 1  lowest_f -1.17779
basinhopping step 2885: f -0.114877 trial_f -0.114877 accepted 1  lowest_f -1.17779
basinhopping step 2886: f -0.114877 trial_f -0.198725 accepted 0  lowest_f -1.17779
basinhopping step 2887: f -0.114877 trial_f -0.150561 accepted 0  lowest_f -1.17779
basinhopping step 2888: f -0.114877 trial_f -0.0975467 accepted 0  lowest_f -1.17779
basinhopping step 2889: f -0.113774 trial_f -0.113774 accepted 1  lowest_f -1.17779
basinhopping step 2890: f -0.0531718 trial_f -0.0531718 accepted 1  lowest_f -1.17779
basinhopping step 2891: f -0.061887 trial_f -0.061887 accepted 1  lowes

basinhopping step 2966: f -0.209583 trial_f -0.209583 accepted 1  lowest_f -1.17779
basinhopping step 2967: f -0.17907 trial_f -0.17907 accepted 1  lowest_f -1.17779
basinhopping step 2968: f -0.200067 trial_f -0.200067 accepted 1  lowest_f -1.17779
basinhopping step 2969: f -0.200067 trial_f -0.127207 accepted 0  lowest_f -1.17779
basinhopping step 2970: f -0.200067 trial_f -0.252904 accepted 0  lowest_f -1.17779
basinhopping step 2971: f -0.200067 trial_f -0.282759 accepted 0  lowest_f -1.17779
basinhopping step 2972: f -0.200067 trial_f -0.236514 accepted 0  lowest_f -1.17779
basinhopping step 2973: f -0.29765 trial_f -0.29765 accepted 1  lowest_f -1.17779
basinhopping step 2974: f -0.251388 trial_f -0.251388 accepted 1  lowest_f -1.17779
basinhopping step 2975: f -0.220719 trial_f -0.220719 accepted 1  lowest_f -1.17779
basinhopping step 2976: f -0.220719 trial_f -0.186826 accepted 0  lowest_f -1.17779
basinhopping step 2977: f -0.318335 trial_f -0.318335 accepted 1  lowest_f -1.17